In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from exp.nb_Tesis import *

In [11]:
path_train_images = Path('./segmented_images/ISIC_2019_Training_Input/')

In [14]:
path_ground_truth = Path('ISIC_2019_Training_GroundTruth.csv')


In [19]:
df = pd.read_csv(path_ground_truth); print(f'{len(df)} images: ')

dfSingleLabel = pd.DataFrame({'image':df['image'] })
labelcols = df.columns[1:len(df.columns)]
dfSingleLabel['label'] = df[list(labelcols)].idxmax(axis='columns')
dfSingleLabel.head()

25331 images: 


,image,label
0,ISIC_0000000,NV
1,ISIC_0000001,NV
2,ISIC_0000002,MEL
3,ISIC_0000003,NV
4,ISIC_0000004,MEL


In [30]:
dfSingleLabel.groupby('label').count()

,image
label,
AK,867
BCC,3323
BKL,2624
DF,239
MEL,4522
NV,12875
SCC,628
VASC,253


In [20]:
labels_for_2018_live_challenge = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC']
def get_label_for_image_name(image_name):
  return list(dfSingleLabel.loc[dfSingleLabel['image']==image_name].label)[0]
def get_label_for_image_path(image_path): return get_label_for_image_name(image_path.name[:-4])

In [48]:
n_of_images_to_keep_per_class = 300
path_reduced_ds = path_train_images / f'/reduced-{n_of_images_to_keep_per_class}'
path_reduced_ds.mkdir(exist_ok=True)

PermissionError: [Errno 13] Permission denied: '/reduced-300'

In [45]:
for row in dfSingleLabel.loc[dfSingleLabel['label']=='NV'].head(n_of_images_to_keep_per_class).itertuples():
    path_img_orig = path_train_images / (row.image + '.jpg')
    !cp $path_img_orig $path_reduced_ds
    break

cp: cannot create regular file '/reduced-300': Permission denied
